In [151]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import sqlite3
import sklearn

In [152]:
pd.options.display.max_columns = None
players = pd.read_csv('players_mejor_3.csv')

In [153]:
players = players[players["best_of"] == 3]

In [154]:
#players = players.dropna(subset=["map_3"])
#esta linea elimina todos los registros que no tienen map_3, es decir que se jugaron y ganaron los dos primeros mapas de corrido
#el problema de esto es que se pierden los dos registros nuevos y deberia queda el que first_map = map1 y second_map = map2

In [155]:
players.drop(columns=['match_id','m1_kills', 'm1_assists', 'm1_deaths', 'm2_kills', 'm2_assists', 'm2_deaths', 'm3_kills', 'm3_assists', 'm3_deaths', 'kills', 'assists', 'deaths', 'hs', 'flash_assists', 'kast', 'adr', 'fkdiff', 'rating','kills_ct', 'deaths_ct', 'kddiff_ct', 'adr_ct','kast_ct','rating_ct','kills_t','deaths_t','kddiff_t','adr_t','kast_t','rating_t', 'kddiff','m1_kills_ct','m1_deaths_ct','m1_kddiff_ct','m1_adr_ct','m1_kast_ct','m1_rating_ct','m1_kills_t','m1_deaths_t','m1_kddiff_t','m1_adr_t','m1_kast_t','m1_rating_t','m2_kills_ct','m2_deaths_ct','m2_kddiff_ct','m2_adr_ct','m2_kast_ct','m2_rating_ct','m2_kills_t','m2_deaths_t','m2_kddiff_t','m2_adr_t','m2_kast_t','m2_rating_t','m3_kills_ct','m3_deaths_ct','m3_kddiff_ct','m3_adr_ct','m3_kast_ct','m3_rating_ct','m3_kills_t','m3_deaths_t','m3_kddiff_t','m3_kast_t','m3_rating_t','m3_adr_t'],axis=1,inplace=True)
#faltaria borrar columnas date y las correspondientes a ids (quedan para control a la hora de armar el dataset nuevo)

In [156]:
players.sample(3)

,date,player_name,team,opponent,country,player_id,event_id,event_name,best_of,map_1,map_2,map_3,m1_hs,m1_flash_assists,m1_kast,m1_kddiff,m1_adr,m1_fkdiff,m1_rating,m2_hs,m2_flash_assists,m2_kast,m2_kddiff,m2_adr,m2_fkdiff,m2_rating,m3_hs,m3_flash_assists,m3_kast,m3_kddiff,m3_adr,m3_fkdiff,m3_rating
116867,2017-11-02,captainMo,TYLOO,HellRaisers,China,5187,3190,SL i-League Invitational Shanghai 2017,3,Cobblestone,Mirage,Nuke,1,1.0,75.0,-4,57.2,0,0.95,9.0,NaN,72.4,-5.0,62.2,0.0,0.91,5.0,0.0,36.4,-9.0,43.2,-3.0,0.48
22178,2019-09-21,Keoz,Syman,Nemiga,Belgium,15117,2832,NEST Pro: EU & CIS Qualifier,3,Mirage,Inferno,Dust2,10,0.0,71.4,-4,74.9,1,1.04,11.0,3.0,67.9,3.0,71.1,2.0,1.09,14.0,0.0,75.0,6.0,93.9,-1.0,1.39
61511,2018-12-03,ryann,MDL All-Star NikoM,MDL All-Star freakazoid,United States,12088,4293,ESEA MDL Season 29 North America All-Star Game,3,Train,Dust2,Inferno,8,0.0,60.9,-6,70.9,1,0.80,13.0,2.0,70.0,-1.0,62.1,-1.0,0.94,14.0,1.0,66.7,-5.0,66.0,0.0,0.85


In [157]:
players.shape

(178728, 33)

In [158]:
# Crear una conexión a una base de datos en memoria
conn = sqlite3.connect(":memory:")

# Guardar los datos en una tabla temporal
players.to_sql("players", conn, if_exists="replace")

# Ejecutar la consulta SQL para filtrar los resultados
query = """
SELECT date, player_name, team, opponent, country, event_name, 
    map_1 AS first_map, m1_hs AS first_map_hs, m1_kast AS first_map_kast, m1_kddiff AS first_map_kddiff, m1_adr AS first_map_adr,
    m1_fkdiff AS first_map_fkdiff, m1_rating AS first_map_rating, 
    map_2 AS second_map, m2_hs AS second_map_hs, m2_kast AS second_map_kast, m2_kddiff AS second_map_kddiff, m2_adr AS second_map_adr,
    m2_fkdiff AS second_map_fkdiff, m2_rating AS second_map_rating
FROM players;
"""

#aca lo que hacemos es para no perder los registros que solo se jugaron dos mapas porque los ganaron de corrido
#si el map3 tiene NaN no creamos el segundo registro
query2 = """
SELECT date, player_name, team, opponent, country, event_name, 
    map_2 AS first_map, m2_hs AS first_map_hs, m2_kast AS first_map_kast, m2_kddiff AS first_map_kddiff, m2_adr AS first_map_adr,
    m2_fkdiff AS first_map_fkdiff, m2_rating AS first_map_rating, 
    map_3 AS second_map, m3_hs AS second_map_hs, m3_kast AS second_map_kast, m3_kddiff AS second_map_kddiff, m3_adr AS second_map_adr,
    m3_fkdiff AS second_map_fkdiff, m3_rating AS second_map_rating
FROM players
WHERE map_3 IS NOT NULL;
"""

# Obtener un dataset con los registros de la primer query
players_nuevo_first = pd.read_sql_query(query, conn)
# Obterner un dataset con los registros de la segunda query
players_nuevo_second = pd.read_sql_query(query2, conn)

conn.close()

# Concatenamos ambos dataset para obtener el dataset final
players_nuevo = pd.concat([players_nuevo_first, players_nuevo_second], ignore_index=True)


In [159]:
players_nuevo.sample(3)

,date,player_name,team,opponent,country,event_name,first_map,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map,second_map_hs,second_map_kast,second_map_kddiff,second_map_adr,second_map_fkdiff,second_map_rating
116890,2017-09-10,prelideN,CK$,aesthetic,Latvia,HyperX GOEXANIMO CS:GO 5x5 #5,Cache,20.0,90.0,16.0,146.6,4.0,2.10,Overpass,6.0,67.9,-4.0,83.3,-2.0,1.00
92621,2018-03-30,NaToSaphiX,Sprout,Heroic,Denmark,Copenhagen Games 2018,Cobblestone,5.0,74.1,1.0,78.8,2.0,1.19,Overpass,14.0,79.3,13.0,118.8,2.0,1.66
156444,2016-07-31,Roc,HG,5POWER,China,SL i-League S2 China Qualifier,Cobblestone,4.0,52.4,-8.0,69.0,-4.0,0.62,Cache,7.0,64.0,-3.0,77.2,1.0,1.03


In [160]:
players_nuevo.shape

(244370, 20)

In [161]:
players_nuevo.drop(columns=['date', 'event_name', 'player_name', 'team', 'opponent'],axis=1,inplace=True)
#evaluar hacer target encoding para la columna team (ver anotaciones y explicacion)

In [162]:
players_nuevo.sample(10)

,country,first_map,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map,second_map_hs,second_map_kast,second_map_kddiff,second_map_adr,second_map_fkdiff,second_map_rating
98585,Latvia,Mirage,6.0,84.2,4.0,93.6,2.0,1.42,Cache,18.0,77.5,7.0,78.0,1.0,1.23
173288,Turkey,Cobblestone,3.0,NaN,5.0,NaN,4.0,1.12,Train,5.0,NaN,-6.0,NaN,1.0,0.74
116386,Singapore,Cache,7.0,79.2,8.0,78.2,3.0,1.37,Mirage,9.0,95.7,9.0,98.9,2.0,1.53
124338,Poland,Overpass,7.0,73.1,-3.0,96.2,0.0,1.09,Train,8.0,59.3,-5.0,70.4,0.0,0.84
36468,Jordan,Inferno,10.0,61.5,0.0,66.6,-1.0,0.98,Dust2,10.0,56.7,0.0,67.5,-2.0,0.91
107952,Canada,Train,12.0,70.8,1.0,66.4,1.0,1.01,Inferno,8.0,87.0,-2.0,60.9,-1.0,1.04
151668,Poland,Cache,11.0,58.3,-6.0,77.0,-2.0,0.91,Train,3.0,55.6,-13.0,36.1,0.0,0.37
65450,Thailand,Inferno,8.0,89.5,19.0,126.8,2.0,2.16,Train,12.0,77.1,4.0,98.3,0.0,1.27
132576,Brazil,Inferno,9.0,80.0,11.0,78.0,2.0,1.34,Overpass,10.0,84.2,12.0,90.0,1.0,1.71
44599,Turkey,Mirage,12.0,65.5,2.0,91.7,3.0,1.18,Cache,7.0,65.0,-1.0,94.3,-3.0,1.02


In [163]:
num_country = players_nuevo['country'].nunique()
num_first_map = players_nuevo['first_map'].nunique()
num_second_map = players_nuevo['second_map'].nunique()

# Imprimir la cantidad de valores distintos
print("Cantidad de valores distintos en la columna 'country':", num_country)
print("Cantidad de valores distintos en la columna 'first_map':", num_first_map)
print("Cantidad de valores distintos en la columna 'second_map':", num_second_map)

Cantidad de valores distintos en la columna 'country': 109
Cantidad de valores distintos en la columna 'first_map': 10
Cantidad de valores distintos en la columna 'second_map': 10


In [164]:
#hacer one hot enconder a country y first_map/second_map:

# creamos las columnas encodeadas y las formateamos
columnas_encoded = pd.get_dummies(players_nuevo, columns=['country', 'first_map', 'second_map'])
columnas_encoded.columns = columnas_encoded.columns.str.lower()

# concatenamos ambos datasets
players_nuevo = pd.concat([players_nuevo, columnas_encoded], axis=1)

# borramos las columnas originales
players_nuevo.drop(['country', 'first_map', 'second_map'], axis=1, inplace=True)


In [165]:
list(players_nuevo.columns)

['first_map_hs',
 'first_map_kast',
 'first_map_kddiff',
 'first_map_adr',
 'first_map_fkdiff',
 'first_map_rating',
 'second_map_hs',
 'second_map_kast',
 'second_map_kddiff',
 'second_map_adr',
 'second_map_fkdiff',
 'second_map_rating',
 'first_map_hs',
 'first_map_kast',
 'first_map_kddiff',
 'first_map_adr',
 'first_map_fkdiff',
 'first_map_rating',
 'second_map_hs',
 'second_map_kast',
 'second_map_kddiff',
 'second_map_adr',
 'second_map_fkdiff',
 'second_map_rating',
 'country_algeria',
 'country_argentina',
 'country_armenia',
 'country_australia',
 'country_austria',
 'country_azerbaijan',
 'country_bahrain',
 'country_bangladesh',
 'country_belarus',
 'country_belgium',
 'country_bolivia',
 'country_bosnia and herzegovina',
 'country_brazil',
 'country_brunei',
 'country_bulgaria',
 'country_cambodia',
 'country_canada',
 'country_chile',
 'country_china',
 'country_colombia',
 'country_costa rica',
 'country_croatia',
 'country_cyprus',
 'country_czech republic',
 'country_

In [94]:
players_nuevo.sample(10)

,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_hs,second_map_kast,second_map_kddiff,second_map_adr,second_map_fkdiff,second_map_rating,first_map_hs,first_map_kast,first_map_kddiff,first_map_adr,first_map_fkdiff,first_map_rating,second_map_hs,second_map_kast,second_map_kddiff,second_map_adr,second_map_fkdiff,second_map_rating,country_albania,country_algeria,country_argentina,country_armenia,country_australia,country_austria,country_azerbaijan,country_bahrain,country_bangladesh,country_belarus,country_belgium,country_bolivia,country_bosnia and herzegovina,country_brazil,country_brunei,country_bulgaria,country_cambodia,country_canada,country_chile,country_china,country_colombia,country_costa rica,country_croatia,country_cyprus,country_czech republic,country_denmark,country_dominican republic,country_ecuador,country_egypt,country_el salvador,country_estonia,country_faroe islands,country_finland,country_france,country_georgia,country_germany,country_greece,country_guatemala,country_hong kong,country_hungary,country_iceland,country_india,country_indonesia,country_iran,country_iraq,country_ireland,country_israel,country_italy,country_japan,country_jordan,country_kazakhstan,country_korea,country_kuwait,country_kyrgyzstan,country_laos,country_latvia,country_lebanon,country_libya,country_lithuania,country_luxembourg,country_macau,country_macedonia,country_malaysia,country_malta,country_mexico,country_moldova,country_mongolia,country_montenegro,country_morocco,country_myanmar,country_nepal,country_netherlands,country_new zealand,country_norway,country_pakistan,country_palestine,country_panama,country_paraguay,country_peru,country_philippines,country_poland,country_portugal,country_qatar,country_romania,country_russia,country_saudi arabia,country_serbia,country_singapore,country_slovakia,country_slovenia,country_south africa,country_spain,country_sri lanka,country_sweden,country_switzerland,country_syria,country_taiwan,country_thailand,country_tunisia,country_turkey,country_turkmenistan,country_ukraine,country_united arab emirates,country_united kingdom,country_united states,country_uruguay,country_uzbekistan,country_venezuela,country_vietnam,first_map_cache,first_map_cobblestone,first_map_default,first_map_dust2,first_map_inferno,first_map_mirage,first_map_nuke,first_map_overpass,first_map_train,first_map_vertigo,second_map_cache,second_map_cobblestone,second_map_default,second_map_dust2,second_map_inferno,second_map_mirage,second_map_nuke,second_map_overpass,second_map_train,second_map_vertigo
180753,5.0,66.7,-11.0,42.5,-2.0,0.64,5.0,92.6,-1.0,79.7,0.0,1.11,5.0,66.7,-11.0,42.5,-2.0,0.64,5.0,92.6,-1.0,79.7,0.0,1.11,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
222853,3.0,29.4,-11.0,38.8,-1.0,0.26,5.0,82.6,-2.0,52.2,1.0,0.95,3.0,29.4,-11.0,38.8,-1.0,0.26,5.0,82.6,-2.0,52.2,1.0,0.95,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,F

In [ ]:
#ver de hacer normalization de algunas columnas (las que tienen valores cercanas a 100 pasarlas a 10)
columnas_a_normalizar = ['first_map_kast', 'first_map_adr', 'second_map_kast', 'second_map_adr']

# Crear un objeto MinMaxScaler para normalizar las columnas
scaler = sklearn.MinMaxScaler()

# Normalizar las columnas seleccionadas
players_nuevo[columnas_normalizadas] = scaler.fit_transform(players_nuevo[columnas_a_normalizar])